In [1]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

In [11]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

In [16]:
def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['경복궁']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)

    driver.quit()
    print("finish")


In [17]:
def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

In [18]:
def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [19]:
def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

##### 0
#### 경복궁
대한민국 수도의 중심이었던 곳. 화려한 장식과 수려한 궁들이 즐비한 이곳. 가끔 노을 보며 구경하면 최고입니다. 이번에 코로나땜에 야간 개장이 중단됬을진 모르겠는 데 시간되면 가보고 싶은 곳.  솔찍히 조선 왕조의 왕궁인니까 5점 드립니다 ㅎㅎ/5
카카오는 반성하라!! 왜 나라를 사랑하는 마음으로써 경복궁에 좋은글을 올렸는데 그걸 또 삭제하고 앉아있냐!!/5
최고예요/5
Really good place /5
서울에 이런 곳이 있다니 너무 감사하네요!!!/5
북악산 인왕산의 멋진 모습을 감상할 수 있어 더 좋은 곳./5
최고예요/5
너무  고생하고  아름다워 ㅇ/5
너무  아름다워요 /5
아름답다/5
궁이 아름답다/5
궁이아름답다/3
궁이 아름답다/4
최고예요/5
최고예요/5
별로예요/1
보통이에요/3
최고예요/5
거대하지 않지만 서울가면 꼭 다녀오는 곳. 하지만 경복궁 갈때마다 볼거리가 엄청나지 않아 약간 아쉬움. /4
좋아여~♡♡/5
최고예요/5
사진찍기도 좋구요. 주변에 화장실이 없어서 경복궁 입장권 끊고 화장실 간적도 있습니다.. /4
웅장하진 않지만 그래도 좋았음/5
경회루 바라보는 벤치 앞에 화장품 샘플 반쯤 묻혀져 있음. 청소는 좀 하세요. 아마 문화재청에서 싸게 용역계약 했나보네. 인건비 줄일라고 청소하는 사람 없앴나 봄. 그리고  오후 4시부터는 안에 있는 사람들 빨리 내 보낼려고 관리용역들이 발악을 함 아예 웬만한 건물안엔 들어가지 말라고 막음. 아니그럼 오후4시에 발권하고 들어온 사람은 뭐 되겠음!! ?? 3.../2
쵝오/5
자금성보다 훨씬 멋있고 아담한 맛이 있다/5
좋아요/4
대한민국의 첫 궁궐은 아닙니다만;;/5
최고예요/5
최고예요/5
자금성보다 훨씬 멋있고 아담한 맛이 있다/5
좋아요/4
대한민국의 첫 궁궐은 아닙니다만;;/5
최고예요/5
최고예요/5


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=85.0.4183.102)


https://github.com/wlgh325/python_crawling